In [2]:
import requests 
import urllib
import pandas as pd

PAGE_SIZE = 500
#DHVXC_REST_DECODED_URL = 'https://de.dhv-xc.de/api/fli/flights?fkcat[]=1&fkto[]={place_id}&navpars={"start":{start},"limit":{limit},"sort":[{"field":"FlightDate","dir":-1},{"field":"BestTaskPoints","dir":-1}]}'

places = {
    'Metzingen':11185,
    'Estorf': 11001,
    'Leese': 10746,
    'Lüdingen':9759,
    'Brunsberg': 9844,
    'Kella': 9521,
    'Börry': 9403,
    'Porta': 9712,    
    'Königszinne': 11489,
    'Rammelsberg': 9427    
}


results = []
#for place in ['Metzingen','Rammelsberg']:
for place in places:
    limit = PAGE_SIZE
    for start in [0, PAGE_SIZE]:
        query = {"navpars":{"start":start,"limit":limit,"sort":[{"field":"FlightDate","dir":-1},{"field":"BestTaskPoints","dir":-1}]}}
        # for some reason, " is replaced by ' and spaces are added which is replaced by +
        decoded_url = f"https://de.dhv-xc.de/api/fli/flights?s?fkcat%5B%5D=1&fkto%5B%5D={places[place]}&{urllib.parse.urlencode(query,quote_via=urllib.parse.quote_plus).replace('%27', '%22').replace('+', '')}"
        #print(decoded_url)
        r = requests.get(decoded_url)
        if r.status_code==200:
            response = r.json()
            df = pd.DataFrame(response['data'])
            #print(df.columns.values)
            results.extend(df[['FlightDate', 'TakeoffWaypointName' , 'Glider' ,'FlightDuration']].values)


df_results = pd.DataFrame(results,columns=['FlightDate', 'Takeoff' , 'Glider' ,'Duration'])
print(len(df_results))
df_results.tail()


10000


,FlightDate,Takeoff,Glider,Duration
9995,2022-08-06,Rammelsberg,U-Double 38,386
9996,2022-08-06,Rammelsberg,Alpha 7 26,238
9997,2022-08-06,Rammelsberg,Ion 5 M,192
9998,2022-08-06,Rammelsberg,Alpha 7 26,191
9999,2022-08-03,Rammelsberg,Maestro 25,1080


In [3]:
import sys
sys.path.insert(0, "../wetter")
from datetime import datetime
from dotenv import load_dotenv
from wetter import get_wind_data,get_rain_data

load_dotenv('../.env')

timeframe_start,timeframe_end = 2023010100,2024123123

df_wind = get_wind_data(timeframe_start,timeframe_end ,13)
df_wind['Date'] = df_wind.apply(lambda row: datetime.strptime(str(row['MESS_DATUM'])[:8], "%Y%m%d").strftime( "%Y-%m-%d"), axis=1)
df_wind.set_index('Date', inplace=True, drop=True)

df_wind.tail()

,MESS_DATUM,strength,direction
Date,,,
2024-09-27,2024092713,8.9,220
2024-09-28,2024092813,7.4,280
2024-09-29,2024092913,1.7,170
2024-09-30,2024093013,6.8,120
2024-10-01,2024100113,3.4,190


In [8]:
from datetime import datetime

wind_dir = {
    0:  'N',
    10: 'N',
    20: 'NNO',
    30: 'NNO',
    40: 'NO',
    50: 'NO',
    60: 'ONO',
    70: 'ONO',
    80: 'O',
    90: 'O',
    100: 'O',
    110: 'OSO',
    120: 'OSO',
    130: 'SO',
    140: 'SO',
    150: 'SSO',
    160: 'SSO',
    170: 'S',
    180: 'S',
    190: 'S',
    200: 'SSW',
    210: 'SSW',
    220: 'SW',
    230: 'SW',
    240: 'WSW',
    250: 'WSW',
    260: 'W',
    270: 'W',
    280: 'W',    
    290: 'WNW',
    300: 'WNW',
    310: 'NW',
    320: 'NW',
    330: 'NNW',
    340: 'NNW',
    350: 'N',
    360: 'N'
}


#df_results[['FlightDate', 'Takeoff']].groupby('Takeoff').aggregate(['min','max'])
df_results = df_results[(df_results['FlightDate']>'2023-01-01') & (df_results['FlightDate']<'2024-10-01')]

df_results['Takeoff'] = df_results.apply(lambda row: row['Takeoff'].split('-')[0].strip() ,axis=1)
df_gr= df_results[['FlightDate', 'Takeoff','Duration']].groupby(by=['Takeoff','FlightDate' ]).aggregate(count=('Duration','count')).reset_index()
#df_gr.tail()
df_pvt = pd.pivot_table(df_gr,index='FlightDate',columns='Takeoff',aggfunc='sum', fill_value=0)
df_pvt.columns = df_pvt.columns.droplevel(0)
df_pvt = df_pvt[['Metzingen','Lüdingen','Leese','Estorf','Brunsberg','Kella Berg','Börry','Porta','Königszinne','Rammelsberg']]
df_pvt['Year'] = df_pvt.apply(lambda row: str(row.name)[:4], axis=1)
df_pvt['Weekday'] = df_pvt.apply(lambda row: datetime.strptime(row.name,'%Y-%m-%d').strftime("%A"), axis=1)
df_pvt['HAJ Wind dir'] = df_pvt.apply(lambda row:wind_dir[df_wind.loc[row.name,'direction']] if df_wind.index.intersection([row.name]).any() else None,axis=1)
df_pvt['HAJ Wind str'] = df_pvt.apply(lambda row:df_wind.loc[row.name,'strength']*3.6 if df_wind.index.intersection([row.name]).any() else None,axis=1)


filter = df_pvt['Weekday'].isin(['Saturday','Sunday'])
df_pvt[(df_pvt['Metzingen']>=0) & (filter)]

Takeoff,Metzingen,Lüdingen,Leese,Estorf,Brunsberg,Kella Berg,Börry,Porta,Königszinne,Rammelsberg,Year,Weekday,HAJ Wind dir,HAJ Wind str
FlightDate,,,,,,,,,,,,,,
2023-01-07,0,0,0,0,0,0,13,0,0,0,2023,Saturday,SSW,25.20
2023-01-08,0,0,0,0,0,0,8,3,0,0,2023,Sunday,SSW,15.84
2023-01-28,0,0,0,0,0,0,0,0,0,2,2023,Saturday,SSW,8.64
2023-02-04,0,0,0,8,0,1,0,0,0,0,2023,Saturday,OSO,8.28
2023-02-19,0,0,0,0,0,0,0,0,0,2,2023,Sunday,WNW,13.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-19,0,0,0,0,0,0,1,0,0,0,2024,Saturday,None,NaN
2024-10-20,0,0,0,0,0,0,16,20,0,0,2024,Sunday,None,NaN
2024-10-26,2,0,0,4,0,0,9,12,0,0,2024,Saturday,None,NaN


In [40]:
from sklearn import ensemble

rf = ensemble.RandomForestRegressor()

df_training, df_test = df_pvt[(filter) & (df_pvt['Year']=='2023')], df_pvt[(filter) & (df_pvt['Year']=='2024')]

yattr,xattr = ['Metzingen'], ['Lüdingen','Leese','Estorf','Brunsberg','Kella Berg','Börry','Porta','Königszinne','Rammelsberg']


rf.fit(df_training[xattr].to_numpy(),df_training[yattr].to_numpy())

df_training['predict'] = df_training.apply(lambda row: rf.predict([row[xattr].to_numpy()])[0] , axis=1)

#df_training[df_training['predict'] != df_training['Metzingen']]

df_test['predict'] =df_test.apply(lambda row: rf.predict([row[xattr].to_numpy()])[0] , axis=1)

df_missed = df_test[(df_test['predict'] >= 5) & (df_test['Metzingen'] == 0)]


print(df_missed[xattr + ['HAJ Wind dir', 'HAJ Wind str','predict']].to_markdown())

C:\Users\Akos Rapp\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\Akos Rapp\AppData\Local\Temp\ipykernel_1900\1262364757.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['predict'] = df_training.apply(lambda row: rf.predict([row[xattr].to_numpy()])[0] , axis=1)


| FlightDate   |   Lüdingen |   Leese |   Estorf |   Brunsberg |   Kella Berg |   Börry |   Porta |   Königszinne |   Rammelsberg | HAJ Wind dir   |   HAJ Wind str |   predict |
|:-------------|-----------:|--------:|---------:|------------:|-------------:|--------:|--------:|--------------:|--------------:|:---------------|---------------:|----------:|
| 2024-01-28   |          0 |       0 |        2 |           0 |            0 |      10 |      14 |             0 |             0 | SSO            |          21.24 |      8.05 |
| 2024-09-07   |         14 |       0 |       17 |           1 |            0 |       0 |       0 |             0 |             0 | O              |           9.36 |     11.71 |
| 2024-09-08   |          0 |       0 |        0 |           0 |            0 |       0 |       0 |             0 |            11 | W              |          14.76 |      6.46 |
| 2024-09-21   |         21 |       0 |        0 |          12 |            6 |       0 |       0 |           

C:\Users\Akos Rapp\AppData\Local\Temp\ipykernel_1900\1262364757.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predict'] =df_test.apply(lambda row: rf.predict([row[xattr].to_numpy()])[0] , axis=1)
